In [135]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import re
import spacy
import json
# gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import re
from textblob import TextBlob


from pprint import pprint


# vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


In [138]:

# preparing the data

def data_length(file):
    # global jsonLength
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    # jsonLength = len(data)
    return len(data)


def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

In [139]:

# print(data_length('./backend/datasets/combined_data.json'))
length = data_length('C:/Hammad Aslam/BS IT (post ADP)/3rd Semester/Capstone Project/Project/backend/datasets/categories/allFiles/computers_laptops.json')
data_array = []
for i in range(0, length):
    data = load_data(
        'C:/Hammad Aslam/BS IT (post ADP)/3rd Semester/Capstone Project/Project/backend/datasets/categories/allFiles/computers_Laptops.json')[i]["reviews"]
    # print(data[0]["review_body"])
    # print(data[i]["review_body"])
    for j in range(0, len(data)):
        data_array.append(data[j]["review_body"])

# print(data_array)

# stopwords = stopwords.words('english')
# print (stopwords)

# lemmatization
len(data_array)


516

In [140]:
# Preprocessing function
def preprocess_text(text):
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    new_record = re.sub(' +', ' ', text)
    # print(new_record)
    doc = nlp(new_record)
    # print(doc)
    newText = []
    for token in doc:
        # if token.pos_ in allowed_postags:
        newText.append(token.lemma_)
        # token_list = [token for token in doc]
        filtered_list = [token for token in doc if not token.is_stop]
        lemmas = [
            f"{token.lemma_}"
            for token in filtered_list
        ]
        # string = ' '.join(lemmas)
    final = " ".join(lemmas)
    print(final)
    return final


In [162]:
# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=2, random_state=0)

# Define the classifier pipeline
classifier = Pipeline([
    ('tfidf', TfidfVectorizer(preprocessor=preprocess_text)),
    ('clf', LogisticRegression())
])
classifier

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(preprocessor=<function preprocess_text at 0x000002B3972E09A0>)),
                ('clf', LogisticRegression())])

In [163]:
# Train the classifier
classifier.fit(X_train, y_train)

# Evaluate on test data
accuracy = classifier.score(X_test, y_test)
print(f"Accuracy: {accuracy}")

# Example usage to filter out words
example_text = "I bought this Compaq A1500 this week and am more than satisfied with it. Simple installation without any problems, excellent photo quality resolution even on plain paper, very good managing software and the best part is, I didn't need any customer support to make it run :).\nVery good product with excellent features, quality and resolution. Believe me it's better than the best available in this price range."
predicted_label = classifier.predict([example_text])
accuracy
predicted_label

great opportunity .
thankful stable job .
happy unstable situation .
good sign .
Accuracy: 0.5
buy Compaq A1500 week satisfied . simple installation problem , excellent photo quality resolution plain paper , good managing software good , need customer support run :) . 
 good product excellent feature , quality resolution . believe well well available price range .


array([1], dtype=int64)

In [164]:
if predicted_label == 1:
    print("Filtered text:", preprocess_text(example_text))
else:
    print("Text passed through:", preprocess_text(example_text))

buy Compaq A1500 week satisfied . simple installation problem , excellent photo quality resolution plain paper , good managing software good , need customer support run :) . 
 good product excellent feature , quality resolution . believe well well available price range .
Filtered text: buy Compaq A1500 week satisfied . simple installation problem , excellent photo quality resolution plain paper , good managing software good , need customer support run :) . 
 good product excellent feature , quality resolution . believe well well available price range .


In [165]:
newText = []
lemText = preprocess_text(example_text)
genText = gen_words(lemText)
newText.append(genText)
newText

buy Compaq A1500 week satisfied . simple installation problem , excellent photo quality resolution plain paper , good managing software good , need customer support run :) . 
 good product excellent feature , quality resolution . believe well well available price range .


[WordList(['compaq week', 'simple installation problem', 'excellent photo quality resolution', 'plain paper good', 'software good need customer support run good product', 'quality resolution', 'available price range'])]

In [166]:
# BIGRAMS AND TRIGRAMS
bigram = gensim.models.Phrases(newText, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[newText], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
bigram_mod

# See trigram example
print(trigram_mod[bigram_mod[newText[0]]])

['compaq week', 'simple installation problem', 'excellent photo quality resolution', 'plain paper good', 'software good need customer support run good product', 'quality resolution', 'available price range']


In [167]:
# removing stopwords and lemmatizing

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [168]:
# Remove Stop Words
data_words_nostops = remove_stopwords(newText)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# data_lemmatized

print(data_lemmatized[:1])

[['compaq', 'week', 'simple', 'installation', 'problem', 'excellent', 'photo', 'quality', 'resolution', 'plain', 'paper', 'good', 'software', 'good', 'need', 'customer', 'support', 'run', 'good', 'product', 'quality', 'resolution', 'available', 'price', 'range']]


In [172]:
id2word = corpora.Dictionary(data_lemmatized)

corpus = []
# for text in newText:
#     a = id2word.doc2bow(text)
#     corpus.append(a)
texts = data_lemmatized

corpus = [id2word.doc2bow(text) for text in texts]

# print(corpus)
print(id2word)

Dictionary<21 unique tokens: ['available', 'compaq', 'customer', 'excellent', 'good']...>


In [173]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('available', 1),
  ('compaq', 1),
  ('customer', 1),
  ('excellent', 1),
  ('good', 3),
  ('installation', 1),
  ('need', 1),
  ('paper', 1),
  ('photo', 1),
  ('plain', 1),
  ('price', 1),
  ('problem', 1),
  ('product', 1),
  ('quality', 2),
  ('range', 1),
  ('resolution', 2),
  ('run', 1),
  ('simple', 1),
  ('software', 1),
  ('support', 1),
  ('week', 1)]]

In [174]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=50,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model

In [175]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
lda_model.print_topics()
# doc_lda

[(0,
  '0.048*"quality" + 0.048*"good" + 0.048*"resolution" + 0.048*"simple" + '
  '0.048*"run" + 0.048*"photo" + 0.048*"software" + 0.048*"support" + '
  '0.048*"excellent" + 0.048*"product"'),
 (1,
  '0.048*"quality" + 0.048*"support" + 0.048*"resolution" + 0.048*"good" + '
  '0.048*"run" + 0.048*"software" + 0.048*"need" + 0.048*"excellent" + '
  '0.048*"problem" + 0.048*"range"'),
 (2,
  '0.048*"good" + 0.048*"quality" + 0.048*"resolution" + 0.048*"photo" + '
  '0.048*"problem" + 0.048*"run" + 0.048*"software" + 0.048*"simple" + '
  '0.048*"excellent" + 0.048*"support"'),
 (3,
  '0.050*"good" + 0.049*"resolution" + 0.048*"quality" + 0.048*"simple" + '
  '0.048*"support" + 0.048*"software" + 0.048*"run" + 0.048*"installation" + '
  '0.048*"photo" + 0.047*"price"'),
 (4,
  '0.048*"good" + 0.048*"resolution" + 0.048*"quality" + 0.048*"photo" + '
  '0.048*"problem" + 0.048*"run" + 0.048*"need" + 0.048*"installation" + '
  '0.048*"support" + 0.048*"available"'),
 (5,
  '0.048*"good" + 0

[(0,
  '0.048*"quality" + 0.048*"good" + 0.048*"resolution" + 0.048*"simple" + 0.048*"run" + 0.048*"photo" + 0.048*"software" + 0.048*"support" + 0.048*"excellent" + 0.048*"product"'),
 (1,
  '0.048*"quality" + 0.048*"support" + 0.048*"resolution" + 0.048*"good" + 0.048*"run" + 0.048*"software" + 0.048*"need" + 0.048*"excellent" + 0.048*"problem" + 0.048*"range"'),
 (2,
  '0.048*"good" + 0.048*"quality" + 0.048*"resolution" + 0.048*"photo" + 0.048*"problem" + 0.048*"run" + 0.048*"software" + 0.048*"simple" + 0.048*"excellent" + 0.048*"support"'),
 (3,
  '0.050*"good" + 0.049*"resolution" + 0.048*"quality" + 0.048*"simple" + 0.048*"support" + 0.048*"software" + 0.048*"run" + 0.048*"installation" + 0.048*"photo" + 0.047*"price"'),
 (4,
  '0.048*"good" + 0.048*"resolution" + 0.048*"quality" + 0.048*"photo" + 0.048*"problem" + 0.048*"run" + 0.048*"need" + 0.048*"installation" + 0.048*"support" + 0.048*"available"'),
 (5,
  '0.048*"good" + 0.048*"quality" + 0.048*"resolution" + 0.048*"photo

In [176]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
coherence_model_lda


Perplexity:  -4.390558569431305

Coherence Score:  0.9999999999999998


In [178]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

TypeError: Object of type complex is not JSON serializable

PreparedData(topic_coordinates=                        x         y  topics  cluster       Freq
topic                                                          
8     -0.011792+0.000000j  0.0-0.0j       1        1  98.827474
3      0.000536+0.000000j  0.0-0.0j       2        1   0.181469
0      0.001407+0.000000j  0.0-0.0j       3        1   0.123883
5      0.001407+0.000000j  0.0+0.0j       4        1   0.123883
4      0.001407+0.000000j -0.0+0.0j       5        1   0.123882
6      0.001407+0.000000j  0.0+0.0j       6        1   0.123882
7      0.001407+0.000000j  0.0-0.0j       7        1   0.123882
1      0.001407+0.000000j  0.0-0.0j       8        1   0.123882
9      0.001407+0.000000j -0.0+0.0j       9        1   0.123882
2      0.001407+0.000000j -0.0+0.0j      10        1   0.123881, topic_info=          Term      Freq     Total Category  logprob  loglift
4         good  2.000000  2.000000  Default  21.0000  21.0000
13     quality  1.000000  1.000000  Default  20.0000  20.0000
15 